In [ ]:
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, ELU, Softmax, Flatten, Conv2D
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

In [ ]:
model_iteration = 0
number_epochs = 10
batch_size = 2
my_lr = 1e-5
easy_mode = True

#dataset_name = "rock_paper_scissors"
dataset_name = "beans"

model_name = "mobilenet"

In [ ]:
train_dataset = tfds.load(dataset_name, split="train", shuffle_files=True)
test_dataset = tfds.load(dataset_name, split="test", shuffle_files=True)

In [ ]:
def tfds_to_numpy(dataset):
    
    train_np_iterator = tfds.as_numpy(dataset)

    train_x = None
    train_y = None

    for elem in train_np_iterator:

        image = elem["image"].reshape(-1, *elem["image"].shape)
        label = elem["label"].reshape(1)
        if train_x is None:
            train_x = image
            train_y = label
        else:
            train_x = np.append(train_x, image, 0)
            train_y = np.append(train_y, label, 0)
        
        
    return train_x, train_y

test_x, test_y = tfds_to_numpy(test_dataset)
print(test_x.shape)
train_x, train_y = tfds_to_numpy(train_dataset)
print(train_x.shape)

In [ ]:
image = train_x[:1]
number_classes = np.max(train_y)+ 1 ##np.max(train_dataset[1]) + 1
input_shape = image.shape[1:]

def initialize_model(number_classes=number_classes, \
        input_shape=input_shape, hidden_dims=64, my_lr=my_lr):

    tf.random.set_seed(13)
    np.random.seed(13)
    
    extractor = tf.keras.applications.MobileNet(\
        input_shape=input_shape, include_top=False, weights="imagenet")
    

    # set to True to also train the feature extraction layers
    extractor.trainable = False
    
    model = Sequential([extractor, \
                        Flatten(), \
                        Dropout(0.25), \
                        Dense(hidden_dims), \
                        ELU(), \
                        Dropout(0.25), \
                        Dense(hidden_dims), \
                        ELU(), \
                        Dense(number_classes),
                        Softmax()
                       ])

    example_output = model(np.random.rand(1,*input_shape))
    
    model.compile(optimizer = 'adam',\
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics = ['accuracy']
                 )

    model.optimizer.lr = my_lr
    
    return model

model = initialize_model()

model.summary()

In [ ]:
# Directory for saving model as SavedModel 
saved_model_dir = os.path.join(\
        "..", "models",  f"{dataset_name}_{model_name}{model_iteration:03}")

if os.path.exists(saved_model_dir):
    while os.path.exists(saved_model_dir):
        model_iteration += 1
        saved_model_dir = os.path.join(\
                "..", "models",  f"{dataset_name}_{model_name}{model_iteration:03}")
else:
    os.system(f"mkdir {saved_model_dir} -p")

# Directory for saving model using BackupAndRestore
saved_backup_dir = os.path.join(\
        "..", "backups",  f"{dataset_name}_{model_name}{model_iteration:03}")

if os.path.exists(saved_backup_dir):
    while os.path.exists(saved_backup_dir):
        model_iteration += 1
        saved_backup_dir = os.path.join(\
                "..", "backups",  f"{dataset_name}_{model_name}{model_iteration:03}")
else:
    os.system(f"mkdir {saved_backup_dir} -p")
    
saved_backup_path = os.path.join(saved_backup_dir, "backup_{epoch:03d}_{val_loss:.2f}.ckpt")
    

# Directory for saving weights (only)
saved_weights_dir = os.path.join(\
        "..", "weights",  f"{dataset_name}_{model_name}{model_iteration:03}")

if os.path.exists(saved_weights_dir):
    while os.path.exists(saved_weights_dir):
        model_iteration += 1
        saved_weights_dir = os.path.join(\
                "..", "weights",  f"{dataset_name}_{model_name}{model_iteration:03}")
else:
    os.system(f"mkdir {saved_weights_dir} -p")

saved_weights_path = os.path.join(saved_weights_dir, "checkpoint_{epoch:03d}_{val_loss:.2f}.hdf5")

In [ ]:
# callbacks
tensorboard_callback = tf.keras.callbacks.TensorBoard(\
    log_dir="logs", \
    write_graph=False, \
    update_freq='epoch', \
)

# This is where we will instantiate a checkpoint callback
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    saved_weights_path, \
    save_freq = "epoch", \
    monitor =  'val_accuracy', \
    save_weights_only = True, \
    verbose = 1)

# This is where we will define a BackupAndRestore callback
backup_callback = tf.keras.callbacks.BackupAndRestore( \
    saved_backup_dir, \
    save_freq = "epoch", \
    delete_checkpoint = True, \
    save_before_preemption = False
    )


# Defining a deliberately deleterious learning rate scheduler
def scheduler(epoch, lr, epochs=number_epochs):
    
    if epoch <= max([1, epochs - epochs // 4]):
        return lr
    else:
        return lr * 20.

# A callback to interrupt training, to demonstrate BackupAndRestore utility
class Interrupt(tf.keras.callbacks.Callback):
    
    def on_epoch_begin(self, epoch, logs=None):
        
        if epoch == 2:
            print("\n Interrupting callback")
            raise RuntimeError("Interrupting callback who?")


lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
interrupt_callback = Interrupt()

basic_callbacks = [tensorboard_callback, lr_callback]

callbacks_with_interrupt = basic_callbacks + [interrupt_callback]
callbacks_with_backup = callbacks_with_interrupt + [backup_callback]

callbacks_with_checkpoints = basic_callbacks + [checkpoint_callback]

In [ ]:
model = initialize_model()

try:
    history = model.fit(x=train_x, y=train_y, \
        validation_split=0.1, \
        batch_size=batch_size, epochs=5, \
        callbacks=callbacks_with_interrupt)
except:
    pass

history = model.fit(x=train_x, y=train_y, \
        validation_split=0.1, \
        batch_size=batch_size, epochs=5, \
        callbacks=basic_callbacks)

In [ ]:
model = initialize_model()

try:
    history = model.fit(x=train_x, y=train_y, \
        validation_split=0.1, \
        batch_size=batch_size, epochs=5, \
        callbacks=callbacks_with_backup)
except:
    pass

history = model.fit(x=train_x, y=train_y, \
        validation_split=0.1, \
        batch_size=batch_size, epochs=5, \
        callbacks=basic_callbacks)

In [ ]:
# Save model weights manually
weights_path = f"manual_weights{model_iteration:03}.ckpt"

model.save_weights(os.path.join(saved_weights_dir, weights_path))

loss, accuracy = model.evaluate(test_x, test_y, \
        batch_size=batch_size)

print(f"model loss: {loss:.3e},  accuracy: {accuracy:.3f}")

model = initialize_model()

loss, accuracy = model.evaluate(test_x, test_y, \
        batch_size=batch_size)


print(f"newly instantiated model loss: {loss:.3e},  accuracy: {accuracy:.3f}")

model.load_weights(os.path.join(saved_weights_dir, weights_path))

loss, accuracy = model.evaluate(test_x, test_y, \
        batch_size=batch_size)

print(f"model (weights loaded from disk) loss: {loss:.3e},  accuracy: {accuracy:.3f}")

In [ ]:
model = initialize_model()

history = model.fit(x=train_x, y=train_y, \
        validation_split=0.1, \
        batch_size=batch_size, epochs=number_epochs, \
        callbacks=callbacks_with_checkpoints)

In [ ]:
# visualize training

my_cmap = plt.get_cmap("magma")
loss_color = my_cmap(16)
val_loss_color = my_cmap(64)

my_cmap = plt.get_cmap("viridis")
acc_color = my_cmap(128)
val_acc_color = my_cmap(192)

fig, ax = plt.subplots(1,1, figsize=(8,4))
ax_twin = ax.twinx()

ax.plot(history.history["loss"], color=loss_color, label="training loss")
ax.plot(history.history["val_loss"], color=val_loss_color, label="validation loss")
ax.set_ylabel("Loss")
ax.set_yticks(np.arange(0,4.0,0.5))

ax_twin.plot(history.history["accuracy"],color=acc_color, label="training accuracy")
ax_twin.plot(history.history["val_accuracy"], color=val_acc_color, label="validation accuracy")
ax_twin.set_ylabel("Accuracy")

ax_twin.set_yticks(np.arange(0,1.0,0.1))
ax.legend(loc=6)

ax_twin.legend(loc=5)
plt.show()

In [ ]:
saved_weights_dir, weights_path, saved_weights_path

In [ ]:
# Restore checkpoint with best performance

# add checkpoint with good performance
good_checkpoint = 6
chkpt_listdir = os.listdir(saved_weights_dir)

for elem in chkpt_listdir:
    if f"checkpoint_{good_checkpoint:03}" in elem and "index" not in elem:
        
        print(elem)
        load_it = elem
        

loss, accuracy = model.evaluate(test_x, test_y, \
        batch_size=batch_size)

print(f"model (final training step) loss: {loss:.3e},  accuracy: {accuracy:.3f}")

good_checkpoint_path = os.path.join(saved_weights_dir, load_it)
model.load_weights(good_checkpoint_path) 

loss, accuracy = model.evaluate(test_x, test_y, \
        batch_size=batch_size)

print(f"model (loaded from last good checkpoint) loss: {loss:.3e},  accuracy: {accuracy:.3f}")

In [ ]:
# Save model weights and architecture in SavedModel format
# If we change the model architecture but try to load the weights we saved before
wrong_model = initialize_model(hidden_dims=256)

print("Trying to load weights, into an architecture that does not match")
wrong_model.load_weights(good_checkpoint_path)


In [ ]:
# To save the full Keras Model object, including architecture and optimizer state

model.save(saved_model_dir)
print("model saved")


loss, accuracy = model.evaluate(test_x, test_y, \
        batch_size=batch_size)
print(f"model that was saved to SavedModel directory, loss: {loss:.3e},  accuracy: {accuracy:.3f}")

restored_model = tf.keras.models.load_model(saved_model_dir)
print("model restored")

loss, accuracy = restored_model.evaluate(test_x, test_y, \
        batch_size=batch_size)
print(f"restored from SavedModel directory, loss: {loss:.3e},  accuracy: {accuracy:.3f}")

In [ ]:
# Convert Keras SavedModel to a TF Lite model

model.save(saved_model_dir)
print(f"model saved to {saved_model_dir}")
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tf_lite_model = converter.convert()

tf_lite_path = os.path.join(\
        "..", "models",  f"{dataset_name}_tf_lite")

# Save the model.
with open(tf_lite_path, "wb") as f:
    f.write(tf_lite_model)

print(f"tf lite model saved to {tf_lite_path}")

interpreter = tf.lite.Interpreter(model_path=tf_lite_path)


signatures = interpreter.get_signature_list()


tf_lite_signature = interpreter.get_signature_runner()
input_details = tf_lite_signature.get_input_details()
output_details = tf_lite_signature.get_output_details()

print(f"TF Lite signatures {signatures}")
print(f"TF Lite input details {input_details}")

In [ ]:
correct_lite = 0
total_samples = test_x.shape[0]

for my_index in range(test_x.shape[0]):

    dtype = input_details[list(input_details.keys())[0]]["dtype"]
    my_batch = np.array(test_x[my_index:my_index+1], dtype=dtype)
    
    full_output_data = model(my_batch)
    input_name = list(input_details.keys())[0] 
    output_name = list(output_details.keys())[0]
    
    output_data = tf_lite_signature(**{input_name: my_batch})[output_name]                   
    
    true_label = test_y[my_index]
    
    correct_lite += 1.0 * (output_data.argmax() == true_label)
    
accuracy_lite = correct_lite / total_samples

print(f"TF Lite test accuracy: {accuracy_lite}")

In [ ]:
# Convert model to TensorFlow JS
# to install tensorflowjs:
# ! pip install tensorflowjs

import tensorflowjs as tfjs

# convert from the keras model directly
tfjs.converters.save_keras_model(model, "tfjs_from_model")

# convert from the SavedModel directory
tfjs.converters.convert_tf_saved_model(saved_model_dir, "tfjs_from_saved_model")

In [ ]:
# Check the contents of the tfjs directories
! ls tfjs_from_model

In [ ]:
! ls tfjs_from_saved_model